In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser
from langchain.callbacks import StreamingStdOutCallbackHandler
import os

# 문서 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

with open("document.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
texts = text_splitter.split_text(raw_text)

# 임베딩 및 벡터 저장소 생성
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(texts, embeddings)

# 검색 함수 정의
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 3}
)

# 프롬프트 템플릿 정의
template = """다음은 1984라는 소설의 일부입니다. 주어진 컨텍스트를 바탕으로 질문에 답변해주세요.

컨텍스트:
{context}

대화 기록:
{chat_history}

질문: {question}

답변:"""

prompt = ChatPromptTemplate.from_template(template)

# 메모리 설정
memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="question",
    return_messages=True
)

# LLM 설정
llm = ChatOpenAI(
    temperature=0,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model="gpt-3.5-turbo",
)

# 문서 검색 함수
def get_context(question):
    docs = retriever.invoke(question)
    return "\n\n".join([doc.page_content for doc in docs])

# RAG 체인 구성
chain = (
    {
        "context": lambda x: get_context(x["question"]),
        "question": lambda x: x["question"],
        "chat_history": lambda x: memory.load_memory_variables({})["chat_history"]
    }
    | prompt
    | llm
    | StrOutputParser()
)

# 실행 함수
def ask(question):
    result = chain.invoke({"question": question})
    memory.save_context({"question": question}, {"output": result})
    return result

# 질문 실행
print("Q: Aaronson은 유죄인가요?")
print("A:", ask("Aaronson은 유죄인가요?"))
print("\nQ: 그가 테이블에 어떤 메시지를 썼나요?")
print("A:", ask("그가 테이블에 어떤 메시지를 썼나요?"))
print("\nQ: Julia는 누구인가요?")
print("A:", ask("Julia는 누구인가요?"))

Q: Aaronson은 유죄인가요?
Aaronson은 유죄로 여겨졌지만, 이야기 속에서는 그의 유죄를 반증하는 사진이 존재하지 않았고, 주인공이 그 사진을 만들어 냈다고 나와 있습니다. 따라서, 사실상 Aaronson이 유죄인지 아닌지는 확실하지 않습니다.A: Aaronson은 유죄로 여겨졌지만, 이야기 속에서는 그의 유죄를 반증하는 사진이 존재하지 않았고, 주인공이 그 사진을 만들어 냈다고 나와 있습니다. 따라서, 사실상 Aaronson이 유죄인지 아닌지는 확실하지 않습니다.

Q: 그가 테이블에 어떤 메시지를 썼나요?
그가 테이블에는 "Aaronson은 유죄로 여겨졌지만, 이야기 속에서는 그의 유죄를 반증하는 사진이 존재하지 않았고, 주인공이 그 사진을 만들어 냈다고 나와 있습니다. 따라서, 사실상 Aaronson이 유죄인지 아닌지는 확실하지 않습니다." 라는 메시지를 썼습니다.A: 그가 테이블에는 "Aaronson은 유죄로 여겨졌지만, 이야기 속에서는 그의 유죄를 반증하는 사진이 존재하지 않았고, 주인공이 그 사진을 만들어 냈다고 나와 있습니다. 따라서, 사실상 Aaronson이 유죄인지 아닌지는 확실하지 않습니다." 라는 메시지를 썼습니다.

Q: Julia는 누구인가요?
Julia는 주인공이 사랑하는 여성이며, 그의 과거의 연인입니다. 현재 이야기 속에서는 주인공이 그녀를 다시 만나고 싶어하는 강한 감정을 느끼고 있습니다.A: Julia는 주인공이 사랑하는 여성이며, 그의 과거의 연인입니다. 현재 이야기 속에서는 주인공이 그녀를 다시 만나고 싶어하는 강한 감정을 느끼고 있습니다.
